In [98]:
import pandas as pd

In [99]:
data = pd.read_csv("result_seq2seq.csv")

In [100]:
data["answer"] = data["answer"].str.replace('_', '')

data

In [101]:
ans = data["answer"]
pred = data["predict"]

In [102]:
ans

0                        まー \r\n
1         うん 運動 好き だっ た ん だ \r\n
2               あー あはは っ はは \r\n
3                      ほうほう \r\n
4                   そう です ね \r\n
                  ...           
22195                    ああ \r\n
22196                    あー \r\n
22197                ビー玉 はい \r\n
22198                    ああ \r\n
22199    え え スキー を やら れ て て \r\n
Name: answer, Length: 22200, dtype: object

# 完全一致率の計算

In [103]:
cnt = 0
for i in range(len(ans)):
    if ans[i] == pred[i]:
        #print("ans:"+ans[i]+" pred:"+pred[i])
        cnt+=1

In [104]:
cnt

1262

In [105]:
same_percentage = (cnt/len(ans))*100

In [106]:
same_percentage

5.684684684684685

# 種類数の計算

In [107]:
col = []
for i in range(len(pred)):
    if pred[i] not in col:
        #print(pred[i])
        col.append(pred[i])

In [108]:
len(col)

49

# BLUEの計算

入力に対する答えが複数あるのですべてに対してスコアを計算する


In [109]:
import nltk

In [110]:
df_ans = data.assign(bleu = 0)

In [111]:
df_ans

,Unnamed: 0,input,answer,predict,bleu
0,0,葬儀 の あいだ じゅう それ から 食事 中 終わっ て から も わたくし の あと ば...,まー \r\n,あー \r\n,0
1,1,一人 で おてんば でし た \r\n,うん 運動 好き だっ た ん だ \r\n,あー そう な ん です か \r\n,0
2,2,泥棒 さん みたい ね って \r\n,あー あはは っ はは \r\n,あー \r\n,0
3,3,つい この 間 も です ね 富士 の 芝桜 \r\n,ほうほう \r\n,うーん \r\n,0
4,4,ただし \r\n,そう です ね \r\n,はい あー \r\n,0
...,...,...,...,...,...
22195,22195,そして 魚 を 獲る と いう よう な \r\n,ああ \r\n,あ \r\n,0
22196,22196,かん と う しゅう だ いれ ん し と いう 風 に \r\n,あー \r\n,あー \r\n,0
22197,22197,ビー玉 \r\n,ビー玉 はい \r\n,あ はい \r\n,0
22198,22198,肩こり が ひどい ん で \r\n,ああ \r\n,あ \r\n,0


In [112]:
df_ans = df_ans.groupby("input").agg({
    'answer':list,
    "predict": list,
    "bleu" : list
})

In [137]:
df_ans = df_ans.reset_index()

In [138]:
df_ans

,input,answer,predict,bleu
0,1 1 時 半 頃 \r\n,"[いらっしゃっ た うーん \r\n, 十 一 時 半 に \r\n]","[あ そう な ん です ね \r\n, あ そう な ん です ね \r\n]","[1.1350838348811001e-231, 1.1350838348811001e-..."
1,1 5 分 ぐらい まで \r\n,"[十 五 分 \r\n, あっ \r\n]","[あー そう な ん です ね \r\n, あー そう な ん です ね \r\n]","[6.091662401187849e-232, 6.091662401187849e-232]"
2,1 5 分 ぐらい まで 駅 に \r\n,"[あ はい \r\n, 1 5 分 はい \r\n]","[はい あー \r\n, はい あー \r\n]","[1.3831258798410648e-231, 1.3831258798410648e-..."
3,1 5 分 ぐらい まで 駅 に 着く まで に かかり まし た \r\n,"[はい だっ た ん です ね ありがとう \r\n, はい うーん うん \r\n, あ ...","[はい あー そう な ん です か \r\n, はい あー そう な ん です か \r\...","[1.2383537059600696e-231, 1.2383537059600696e-..."
4,1 5 分 前 に \r\n,"[凄い です \r\n, あっ \r\n, や \r\n, 予定 通り ああ \r\n]","[はい あー そう な ん です ね \r\n, はい あー そう な ん です ね \r\...","[6.506563597699589e-232, 6.506563597699589e-23..."
...,...,...,...,...
7509,鳥 を 捕まえる と いう \r\n,"[うん \r\n, えー \r\n, 捕まえる ん です か \r\n, へー \r\n, ...","[はー \r\n, はー \r\n, はー \r\n, はー \r\n, はー \r\n, ...","[2.5532866485320654e-232, 2.5532866485320654e-..."
7510,鳥 を 捕まえる と いう そんな \r\n,"[へえ \r\n, あ はい そう です ね \r\n, 鳥 を \r\n, 鳥 \r\n,...","[はー \r\n, はー \r\n, はー \r\n, はー \r\n, はー \r\n, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7511,麹町 番長 の \r\n,[そう な ん です ね \r\n],[あー そう な ん です ね \r\n],[1.4875195904069663e-231]
7512,麹町 番長 の 大きな お 屋敷 の おー 石造り の バルコニー みたい な 部分 だけ ...,"[そう な ん です か \r\n, はー \r\n, ああ \r\n, あー \r\n]","[あー \r\n, あー \r\n, あー \r\n, あー \r\n]","[3.829929972798098e-232, 3.829929972798098e-23..."


In [139]:
from janome.tokenizer import Tokenizer
j_t = Tokenizer()

def tokenizer(text):
    return [tok for tok in j_t.tokenize(text, wakati = True)]

In [140]:
len(df_ans)

7514

In [141]:
col = []
for k in range(len(df_ans)):
    for j in range(len(df_ans["predict"][k])):
        for i in range(len(df_ans["answer"][k])):
            src = df_ans["answer"][k][i]
            ans = df_ans["predict"][k][j]
            src = tokenizer(src)
            ans = tokenizer(ans)
            col.append(nltk.translate.bleu_score.sentence_bleu(ans, src))
        #print(col)    
        ave = sum(col)/len(col)
        #print(ave)
        df_ans["bleu"][k][j] = ave
        col = []
    

C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

[1.2183324802375697e-231, 1.0518351895246305e-231]
1.1350838348811001e-231
[1.2183324802375697e-231, 1.0518351895246305e-231]
1.1350838348811001e-231
[1.2183324802375697e-231, 0]
6.091662401187849e-232
[1.2183324802375697e-231, 0]
6.091662401187849e-232
[1.646211035903463e-231, 1.1200407237786664e-231]
1.3831258798410648e-231
[1.646211035903463e-231, 1.1200407237786664e-231]
1.3831258798410648e-231
[1.1409851298103347e-231, 1.2183324802375697e-231, 1.4740564900137075e-231, 1.1200407237786664e-231]
1.2383537059600696e-231
[1.1409851298103347e-231, 1.2183324802375697e-231, 1.4740564900137075e-231, 1.1200407237786664e-231]
1.2383537059600696e-231
[1.1409851298103347e-231, 1.2183324802375697e-231, 1.4740564900137075e-231, 1.1200407237786664e-231]
1.2383537059600696e-231
[1.1409851298103347e-231, 1.2183324802375697e-231, 1.4740564900137075e-231, 1.1200407237786664e-231]
1.2383537059600696e-231
[1.384292958842266e-231, 0, 0, 1.2183324802375697e-231]
6.506563597699589e-232
[1.384292958842266e

[0]
0.0
[1.821831989445342e-231]
1.821831989445342e-231
[0]
0.0
[0]
0.0
[1.1200407237786664e-231, 1.2183324802375697e-231]
1.1691866020081181e-231
[1.1200407237786664e-231, 1.2183324802375697e-231]
1.1691866020081181e-231
[1.0518351895246305e-231, 0]
5.2591759476231526e-232
[1.0518351895246305e-231, 0]
5.2591759476231526e-232
[1.2183324802375697e-231, 1.646211035903463e-231, 1.1200407237786664e-231, 1.0518351895246305e-231]
1.2591048573610824e-231
[1.2183324802375697e-231, 1.646211035903463e-231, 1.1200407237786664e-231, 1.0518351895246305e-231]
1.2591048573610824e-231
[1.2183324802375697e-231, 1.646211035903463e-231, 1.1200407237786664e-231, 1.0518351895246305e-231]
1.2591048573610824e-231
[1.2183324802375697e-231, 1.646211035903463e-231, 1.1200407237786664e-231, 1.0518351895246305e-231]
1.2591048573610824e-231
[1.646211035903463e-231, 1.0518351895246305e-231]
1.3490231127140468e-231
[1.646211035903463e-231, 1.0518351895246305e-231]
1.3490231127140468e-231
[1.4488496539373276e-231, 0,

[0, 0, 1.1200407237786664e-231, 1.1200407237786664e-231, 1.0518351895246305e-231, 1.384292958842266e-231, 1.0518351895246305e-231, 1.0832677820940877e-231, 1.2183324802375697e-231, 1.384292958842266e-231, 1.384292958842266e-231]
9.816573604968226e-232
[0, 0, 1.1200407237786664e-231, 1.1200407237786664e-231, 1.0518351895246305e-231, 1.384292958842266e-231, 1.0518351895246305e-231, 1.0832677820940877e-231, 1.2183324802375697e-231, 1.384292958842266e-231, 1.384292958842266e-231]
9.816573604968226e-232
[0, 0, 1.1200407237786664e-231, 1.1200407237786664e-231, 1.0518351895246305e-231, 1.384292958842266e-231, 1.0518351895246305e-231, 1.0832677820940877e-231, 1.2183324802375697e-231, 1.384292958842266e-231, 1.384292958842266e-231]
9.816573604968226e-232
[0]
0.0
[1.4875195904069663e-231]
1.4875195904069663e-231
[1.646211035903463e-231, 1.0244914152188952e-231, 9.788429383461836e-232, 0, 1.1200407237786664e-231]
9.539172226494416e-232
[1.646211035903463e-231, 1.0244914152188952e-231, 9.788429383

[0, 0, 0, 0]
0.0
[0, 0, 0, 0]
0.0
[0, 0, 0, 0]
0.0
[0, 0, 0, 0]
0.0
[1.0003688322288243e-231, 1.2183324802375697e-231, 1.2882297539194154e-231]
1.1689770221286031e-231
[1.0003688322288243e-231, 1.2183324802375697e-231, 1.2882297539194154e-231]
1.1689770221286031e-231
[1.0003688322288243e-231, 1.2183324802375697e-231, 1.2882297539194154e-231]
1.1689770221286031e-231
[1.0518351895246305e-231, 0]
5.2591759476231526e-232
[1.0518351895246305e-231, 0]
5.2591759476231526e-232
[0]
0.0
[1.1200407237786664e-231, 1.0518351895246305e-231]
1.0859379566516484e-231
[1.1200407237786664e-231, 1.0518351895246305e-231]
1.0859379566516484e-231
[0, 0]
0.0
[0, 0]
0.0
[0]
0.0
[0]
0.0
[1.0518351895246305e-231, 1.646211035903463e-231, 1.1008876702055895e-231]
1.266311298544561e-231
[1.0518351895246305e-231, 1.646211035903463e-231, 1.1008876702055895e-231]
1.266311298544561e-231
[1.0518351895246305e-231, 1.646211035903463e-231, 1.1008876702055895e-231]
1.266311298544561e-231
[0]
0.0
[1.821831989445342e-231]
1.8

...

[1.2508498911928379e-231, 0, 1.4740564900137075e-231, 0, 1.1409851298103347e-231, 1.0518351895246305e-231]
8.196211167569184e-232
[1.2508498911928379e-231, 0, 1.4740564900137075e-231, 0, 1.1409851298103347e-231, 1.0518351895246305e-231]
8.196211167569184e-232
[1.2508498911928379e-231, 0, 1.4740564900137075e-231, 0, 1.1409851298103347e-231, 1.0518351895246305e-231]
8.196211167569184e-232
[1.2508498911928379e-231, 0, 1.4740564900137075e-231, 0, 1.1409851298103347e-231, 1.0518351895246305e-231]
8.196211167569184e-232
[1.2508498911928379e-231, 0, 1.4740564900137075e-231, 0, 1.1409851298103347e-231, 1.0518351895246305e-231]
8.196211167569184e-232
[1.646211035903463e-231, 8.972141065609098e-232, 1.1896457329133973e-231]
1.2443569584592566e-231
[1.646211035903463e-231, 8.972141065609098e-232, 1.1896457329133973e-231]
1.2443569584592566e-231
[1.646211035903463e-231, 8.972141065609098e-232, 1.1896457329133973e-231]
1.2443569584592566e-231
[0, 0, 1.3568676344828118e-231, 0, 1.384292958842266e-23

[9.594503055152632e-232, 0, 0, 1.0832677820940877e-231, 0, 9.291879812217675e-232]
4.953176781385198e-232
[1.1200407237786664e-231, 0, 1.1200407237786664e-231, 1.4740564900137075e-231, 0, 1.4147351699132998e-231, 1.384292958842266e-231, 1.4147351699132998e-231, 1.1200407237786664e-231]
1.0053268844465081e-231
[1.1200407237786664e-231, 0, 1.1200407237786664e-231, 1.4740564900137075e-231, 0, 1.4147351699132998e-231, 1.384292958842266e-231, 1.4147351699132998e-231, 1.1200407237786664e-231]
1.0053268844465081e-231
[1.1200407237786664e-231, 0, 1.1200407237786664e-231, 1.4740564900137075e-231, 0, 1.4147351699132998e-231, 1.384292958842266e-231, 1.4147351699132998e-231, 1.1200407237786664e-231]
1.0053268844465081e-231
[1.1200407237786664e-231, 0, 1.1200407237786664e-231, 1.4740564900137075e-231, 0, 1.4147351699132998e-231, 1.384292958842266e-231, 1.4147351699132998e-231, 1.1200407237786664e-231]
1.0053268844465081e-231
[1.1200407237786664e-231, 0, 1.1200407237786664e-231, 1.4740564900137075e-

[0, 0, 1.384292958842266e-231, 0, 1.1200407237786664e-231]
5.008667365241865e-232
[0, 0, 1.384292958842266e-231, 0, 1.1200407237786664e-231]
5.008667365241865e-232
[0, 0, 1.384292958842266e-231, 0, 1.1200407237786664e-231]
5.008667365241865e-232
[0, 0, 1.384292958842266e-231, 0, 1.1200407237786664e-231]
5.008667365241865e-232
[0]
0.0
[0, 0, 1.5319719891192393e-231]
5.106573297064131e-232
[0, 0, 1.5319719891192393e-231]
5.106573297064131e-232
[0, 0, 1.5319719891192393e-231]
5.106573297064131e-232
[1.384292958842266e-231, 1.2183324802375697e-231, 1.646211035903463e-231, 1.646211035903463e-231, 0, 0, 0, 0, 1.0518351895246305e-231]
7.718758556012658e-232
[1.384292958842266e-231, 1.2183324802375697e-231, 1.646211035903463e-231, 1.646211035903463e-231, 0, 0, 0, 0, 1.0518351895246305e-231]
7.718758556012658e-232
[1.384292958842266e-231, 1.2183324802375697e-231, 1.646211035903463e-231, 1.646211035903463e-231, 0, 0, 0, 0, 1.0518351895246305e-231]
7.718758556012658e-232
[1.384292958842266e-231, 

[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[0, 0, 1.4488496539373276e-231, 0, 0, 0, 1.4488496539373276e-231, 1.646211035903463e-231]
5.6798879297226475e-232
[1.4875195904069663e-231, 0, 1.4488496539373276e-231, 1.4488496539373276e-231, 0, 1.4488

[1.0244914152188952e-231, 1.1200407237786664e-231, 0, 1.384292958842266e-231, 1.1808001696991507e-231, 1.5319719891192393e-231, 1.3165594234639305e-231]
1.0797366685888783e-231
[1.0244914152188952e-231, 1.1200407237786664e-231, 0, 1.384292958842266e-231, 1.1808001696991507e-231, 1.5319719891192393e-231, 1.3165594234639305e-231]
1.0797366685888783e-231
[1.0244914152188952e-231, 1.1200407237786664e-231, 0, 1.384292958842266e-231, 1.1808001696991507e-231, 1.5319719891192393e-231, 1.3165594234639305e-231]
1.0797366685888783e-231
[1.0244914152188952e-231, 1.1200407237786664e-231, 0, 1.384292958842266e-231, 1.1808001696991507e-231, 1.5319719891192393e-231, 1.3165594234639305e-231]
1.0797366685888783e-231
[1.0244914152188952e-231, 1.1200407237786664e-231, 0, 1.384292958842266e-231, 1.1808001696991507e-231, 1.5319719891192393e-231, 1.3165594234639305e-231]
1.0797366685888783e-231
[1.0244914152188952e-231, 1.1200407237786664e-231, 0, 1.384292958842266e-231, 1.1808001696991507e-231, 1.5319719891

In [142]:
df_ans

,input,answer,predict,bleu
0,1 1 時 半 頃 \r\n,"[いらっしゃっ た うーん \r\n, 十 一 時 半 に \r\n]","[あ そう な ん です ね \r\n, あ そう な ん です ね \r\n]","[1.1350838348811001e-231, 1.1350838348811001e-..."
1,1 5 分 ぐらい まで \r\n,"[十 五 分 \r\n, あっ \r\n]","[あー そう な ん です ね \r\n, あー そう な ん です ね \r\n]","[6.091662401187849e-232, 6.091662401187849e-232]"
2,1 5 分 ぐらい まで 駅 に \r\n,"[あ はい \r\n, 1 5 分 はい \r\n]","[はい あー \r\n, はい あー \r\n]","[1.3831258798410648e-231, 1.3831258798410648e-..."
3,1 5 分 ぐらい まで 駅 に 着く まで に かかり まし た \r\n,"[はい だっ た ん です ね ありがとう \r\n, はい うーん うん \r\n, あ ...","[はい あー そう な ん です か \r\n, はい あー そう な ん です か \r\...","[1.2383537059600696e-231, 1.2383537059600696e-..."
4,1 5 分 前 に \r\n,"[凄い です \r\n, あっ \r\n, や \r\n, 予定 通り ああ \r\n]","[はい あー そう な ん です ね \r\n, はい あー そう な ん です ね \r\...","[6.506563597699589e-232, 6.506563597699589e-23..."
...,...,...,...,...
7509,鳥 を 捕まえる と いう \r\n,"[うん \r\n, えー \r\n, 捕まえる ん です か \r\n, へー \r\n, ...","[はー \r\n, はー \r\n, はー \r\n, はー \r\n, はー \r\n, ...","[2.5532866485320654e-232, 2.5532866485320654e-..."
7510,鳥 を 捕まえる と いう そんな \r\n,"[へえ \r\n, あ はい そう です ね \r\n, 鳥 を \r\n, 鳥 \r\n,...","[はー \r\n, はー \r\n, はー \r\n, はー \r\n, はー \r\n, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
7511,麹町 番長 の \r\n,[そう な ん です ね \r\n],[あー そう な ん です ね \r\n],[1.4875195904069663e-231]
7512,麹町 番長 の 大きな お 屋敷 の おー 石造り の バルコニー みたい な 部分 だけ ...,"[そう な ん です か \r\n, はー \r\n, ああ \r\n, あー \r\n]","[あー \r\n, あー \r\n, あー \r\n, あー \r\n]","[3.829929972798098e-232, 3.829929972798098e-23..."


In [143]:
def flatten(row):
    df1 = pd.Series(row['answer']).to_frame(name = 'answer')
    df2 = pd.Series(row['predict']).to_frame(name = 'predict')
    df3 = pd.Series(row['bleu']).to_frame(name = 'bleu')
    df4 = pd.DataFrame([{'src':row['input']}])
    df1['key'] = df2['key'] = df3['key'] = df4['key'] = 0
    return df4.merge(df1, how = 'outer').merge(df2, how='outer').merge(df3, how = 'outer').drop(columns = 'key')

In [ ]:
pd.concat(df_ans.apply(flatten, axis=1).values)

In [ ]:
df_ans